In [1]:

!pip install pygame
import pygame
!pip install opencv-python --user
import cv2
import numpy as np
import pygame
from pygame import mixer
!pip install playsound
from playsound import playsound
import os
!pip install twilio
from twilio.rest import Client 
pygame.mixer.init()

# Define your Twilio credentials
account_sid = 'AC230b0cfadb30a728d6767cfe13465444'
auth_token = '5eee5c61fbfdf0cc38850404b2c92a78'
twilio_phone_number = '+15675295689 '
user_phone_number = '+919989913857'

# Define the file paths for the sound files
crackers_sound_file = "C:\\Users\\91814\\Downloads\\CrackersSound.mp3"
lion_roar_sound_file = "C:\\Users\\91814\\Downloads\\LionRoarSound.mp3"

# Initialize the Twilio client
client = Client(account_sid, auth_token)

cap = cv2.VideoCapture(0)
wht = 320
confThreshold = 0.5
nmsThreshold = 0.3
classesFile = 'coco.names'
classNames = []
with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
modelConfi = 'yolov3.cfg'
modelwei = 'yolov3.weights'
net = cv2.dnn.readNetFromDarknet(modelConfi, modelwei)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

def send_notification(animal):
    message = client.messages.create(
        body=f"An {animal} has entered the field!",
        from_=twilio_phone_number,
        to=user_phone_number
    )
    print(f"Notification sent: {message.sid}")

def play_sound(sound_file):
    try:
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play()
    except pygame.error as e:
        print(f"Error playing sound: {e}")

def findobj(outputs, img):
    ht, wt, ct = img.shape
    bbox = []
    classIds = []
    confs = []

    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w, h = int(det[2] * wt), int(det[3] * ht)
                x, y = int((det[0] * wt) - w / 2), int((det[1] * ht) - h / 2)
                bbox.append([x, y, w, h])
                classIds.append(classId)
                confs.append(float(confidence))
    indices = cv2.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)
    for i in indices:
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 2)
        cv2.putText(img, f'{classNames[classIds[i]].upper()} {int(confs[i] * 100)}%', (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 2))
        x = classNames[classIds[i]]
        if x in ["bird", "horse", "truck", "elephant", "bear", "zebra", "giraffe"]:
            send_notification(x)
            play_sound(crackers_sound_file)
        elif x in ["cat", "dog", "sheep"]:
            send_notification(x)
            play_sound(lion_roar_sound_file)

while True:
    outputnames = []
    success, img = cap.read()
    blob = cv2.dnn.blobFromImage(img, 1/255, (wht, wht), [0, 0, 0], 1, crop=False)
    net.setInput(blob)
    layer_names = net.getLayerNames()
    outputnames = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(outputnames)
    findobj(outputs, img)
    cv2.imshow('image', img)
    k = cv2.waitKey(50)
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Notification sent: SMf56b87b36546d21ee03036548b5c5e54
Notification sent: SMe49c70194c40b8de0c98446ba62bfa81
Notification sent: SMd10f173302750edf470625f5d61a1a00
Notification sent: SM201e19a03e001a2a057b8c8ead377ddf
Notification sent: SM9fd591868841ff7fb2b4e70cdb0206e1
Notification sent: SM9fcf39cbea20b0e474289cc1675360b4
Notification sent: SMf936acda5321c3d47c32141fc849706f
Notification sent: SM197bb8b2f3cdaac09b84a309176faa7e
Notification sent: SMa77832317cb452fc0dc48d4e0f641ab5
Notification sent: SM7a7601e016d92353cdcebc001a2ec750
Notification sent: SMcd24129fe6da96a07685f59bf63baca7
Notification sent: SMd47e27a4935b1eb16fdc2eb0fce2c865
Notification sent: SM525b15185f5ef7bb2ff40ebb2eb8e383
